In [1]:
import numpy as np
import cv2
import os
import io
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import datetime
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from time import time
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, LambdaCallback
from keras.layers import Input, Dropout, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.applications.resnet import ResNet50

In [12]:
def overfit(history):
  # Get the final training and validation losses
  final_train_loss = history.history['loss'][-1]
  final_val_loss = history.history['val_loss'][-1]

  # Print the final losses
  print(f'Final Training Loss: {final_train_loss}')
  print(f'Final Validation Loss: {final_val_loss}')

  # Check for overfitting
  if final_val_loss > final_train_loss:
      print('The model is overfitting.')
  else:
      print('The model is not overfitting.')

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
!kaggle datasets download -d mohamedhanyyy/chest-ctscan-images


Dataset URL: https://www.kaggle.com/datasets/mohamedhanyyy/chest-ctscan-images
License(s): ODbL-1.0
 95% 113M/119M [00:01<00:00, 120MB/s] 
100% 119M/119M [00:01<00:00, 98.9MB/s]


In [2]:
import zipfile
zip_file_path = '/content/chest-ctscan-images.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')


In [3]:
from datetime import datetime
import time

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, Dense, GlobalAveragePooling2D, BatchNormalization, Flatten

import numpy as np
import cv2
import os
import io
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import datetime
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from time import time
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, LambdaCallback
from keras.layers import Input, Dropout, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.applications.resnet import ResNet50

In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Define image size
image_size = 150

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, 0)
    image = cv2.bilateralFilter(image, 2, 50, 50)
    image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
    image = cv2.resize(image, (image_size, image_size))
    return image

# Define data directories
data_dirs = {
    'train': '/content/Data/train',
    'test': '/content/Data/test',
    'valid': '/content/Data/valid'
}

# Initialize lists to store data
data = {split: {'images': [], 'labels': []} for split in data_dirs}

# Iterate over data directories
for split, directory in data_dirs.items():
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for file in tqdm(os.listdir(label_dir), desc=f'Processing {split}/{label}'):
            image_path = os.path.join(label_dir, file)
            data[split]['images'].append(preprocess_image(image_path))
            data[split]['labels'].append(label)

# Convert lists to NumPy arrays and normalize images
for split in data:
    data[split]['images'] = np.array(data[split]['images']) / 255.0
    data[split]['labels'] = np.array(data[split]['labels'])

# Extract unique labels present in the data
unique_labels = np.unique(np.concatenate([data[split]['labels'] for split in data]))

# Convert labels to one-hot encoding using dynamically extracted labels
label_to_index = {label: i for i, label in enumerate(unique_labels)}
for split in data:
    data[split]['labels'] = tf.keras.utils.to_categorical([label_to_index[label] for label in data[split]['labels']], num_classes=len(unique_labels))


# Shuffle training data
data['train']['images'], data['train']['labels'] = shuffle(data['train']['images'], data['train']['labels'], random_state=42)

# Split dataset into training, validation, and test sets
train_val_split = 0.2
val_test_split = 0.5
x_train, x_val_test, y_train, y_val_test = train_test_split(data['train']['images'], data['train']['labels'], test_size=train_val_split, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=val_test_split, random_state=42)



Processing train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib: 100%|██████████| 195/195 [00:01<00:00, 150.89it/s]
Processing train/normal: 100%|██████████| 148/148 [00:01<00:00, 80.36it/s]
Processing train/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa: 100%|██████████| 115/115 [00:00<00:00, 163.83it/s]
Processing train/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa: 100%|██████████| 155/155 [00:00<00:00, 160.99it/s]
Processing test/normal: 100%|██████████| 54/54 [00:01<00:00, 52.83it/s]
Processing valid/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib: 100%|██████████| 23/23 [00:00<00:00, 142.31it/s]
Processing valid/normal: 100%|██████████| 13/13 [00:00<00:00, 37.86it/s]
Processing valid/large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa: 100%|██████████| 21/21 [00:00<00:00, 128.73it/s]
Processing valid/squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa: 100%|██████████| 15/15 [00:00<00:00, 117.66it/s]


In [5]:

import os
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Define image size
image_size = 150

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, 0)
    image = cv2.bilateralFilter(image, 2, 50, 50)
    image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
    image = cv2.resize(image, (image_size, image_size))
    return image

# Define data directories
data_dirs = {
    'train': '/content/Data/train',
    'test': '/content/Data/test',
    'valid': '/content/Data/valid'
}

# Initialize lists to store data
data = {split: {'images': [], 'labels': []} for split in data_dirs}

# Iterate over data directories
for split, directory in data_dirs.items():
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for file in tqdm(os.listdir(label_dir), desc=f'Processing {split}/{label}'):
            image_path = os.path.join(label_dir, file)
            data[split]['images'].append(preprocess_image(image_path))
            data[split]['labels'].append(label)

# Convert lists to NumPy arrays and normalize images
for split in data:
    data[split]['images'] = np.array(data[split]['images']) / 255.0
    data[split]['labels'] = np.array(data[split]['labels'])

# Extract unique labels present in the data
unique_labels = np.unique(np.concatenate([data[split]['labels'] for split in data]))

# Convert labels to one-hot encoding using dynamically extracted labels
label_to_index = {label: i for i, label in enumerate(unique_labels)}
for split in data:
    data[split]['labels'] = tf.keras.utils.to_categorical([label_to_index[label] for label in data[split]['labels']], num_classes=len(unique_labels))

# Shuffle training data
data['train']['images'], data['train']['labels'] = shuffle(data['train']['images'], data['train']['labels'], random_state=42)

# Split dataset into training, validation, and test sets
train_val_split = 0.2
val_test_split = 0.5
x_train, x_val_test, y_train, y_val_test = train_test_split(data['train']['images'], data['train']['labels'], test_size=train_val_split, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=val_test_split, random_state=42)

# Apply data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
datagen.fit(x_train)



Processing train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib:  75%|███████▌  | 147/195 [00:01<00:00, 79.59it/s]


KeyboardInterrupt: 

In [9]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.4 MB/s eta 0:00:00


In [8]:
num_classes = len(unique_labels)

In [11]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import kerastuner as kt
import numpy as np
from time import time

# Define Monte Carlo Dropout CNN model
def build_model(hp):
    inputs = layers.Input(shape=(150, 150, 3))

    # Convolutional layers
    x = layers.Conv2D(64, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(256, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Flatten layer
    x = layers.Flatten()(x)

    # Dense layers with Monte Carlo Dropout
    for i in range(hp.Int('num_layers', min_value=1, max_value=3, step=1)):
        x = layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                         activation='relu')(x)
        x = layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1))(x)

    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

    return model

# Define a tuner
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='keras_tuner_dir',
                     project_name='monte_carlo_cnn')

# Search for the best hyperparameters
tuner.search(x_train, y_train,
             epochs=10,
             validation_data=(x_val, y_val))

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Fit the best model
start_time = time()
history = best_model.fit(x_train, y_train,
                         epochs=10,
                         validation_data=(x_val, y_val))
end_time = time()

# Evaluate the best model on test data using Monte Carlo Dropout
mc_dropout_model = tf.keras.models.Sequential([best_model])
mc_dropout_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                         metrics=['accuracy'])

mc_dropout_predictions = mc_dropout_model.predict(x_test, batch_size=None, verbose=1)
mc_dropout_accuracy = np.mean(np.argmax(mc_dropout_predictions, axis=-1) == np.argmax(y_test, axis=-1))

# Print metrics
print("Monte Carlo Dropout Test Accuracy:", mc_dropout_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Monte Carlo Dropout Test Acc: {mc_dropout_accuracy}')

Trial 30 Complete [00h 00m 23s]
val_accuracy: 0.9016393423080444

Best val_accuracy So Far: 0.9016393423080444
Total elapsed time: 00h 07m 05s
Epoch 1/10
16/16 [==============================] - 4s 75ms/step - loss: 0.1746 - accuracy: 0.9429 - val_loss: 0.4521 - val_accuracy: 0.8852
Epoch 2/10
16/16 [==============================] - 1s 57ms/step - loss: 0.0695 - accuracy: 0.9816 - val_loss: 0.4555 - val_accuracy: 0.8689
Epoch 3/10
16/16 [==============================] - 1s 59ms/step - loss: 0.0341 - accuracy: 0.9857 - val_loss: 0.4933 - val_accuracy: 0.8689
Epoch 4/10
16/16 [==============================] - 1s 61ms/step - loss: 0.0123 - accuracy: 1.0000 - val_loss: 0.6661 - val_accuracy: 0.8852
Epoch 5/10
16/16 [==============================] - 1s 56ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.6676 - val_accuracy: 0.8852
Epoch 6/10
16/16 [==============================] - 1s 57ms/step - loss: 0.0066 - accuracy: 0.9980 - val_loss: 0.6209 - val_accuracy: 0.9180
Epoch 7/10


In [12]:
overfit(history)

Final Training Loss: 0.03339618071913719
Final Validation Loss: 0.4890919327735901
The model is overfitting.


In [17]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from time import time

# Define Monte Carlo Dropout CNN model
def build_model():
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # Adjust regularization strength
        layers.Dropout(0.8),  # Increase dropout rate
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Instantiate the model
model = build_model()

# Data Augmentation (even more aggressive)
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with more aggressive data augmentation
start_time = time()
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) // 32,
                    epochs=15,  # Increase epochs for more training
                    validation_data=(x_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])  # Adjust patience

end_time = time()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')


Epoch 1/15
15/15 [==============================] - 7s 217ms/step - loss: 4.1159 - accuracy: 0.2882 - val_loss: 2.6271 - val_accuracy: 0.2295
Epoch 2/15
15/15 [==============================] - 4s 242ms/step - loss: 2.3018 - accuracy: 0.3188 - val_loss: 1.7295 - val_accuracy: 0.4426
Epoch 3/15
15/15 [==============================] - 3s 172ms/step - loss: 1.6596 - accuracy: 0.4521 - val_loss: 1.5317 - val_accuracy: 0.4426
Epoch 4/15
15/15 [==============================] - 2s 163ms/step - loss: 1.5587 - accuracy: 0.4389 - val_loss: 1.3899 - val_accuracy: 0.4426
Epoch 5/15
15/15 [==============================] - 4s 250ms/step - loss: 1.3275 - accuracy: 0.4803 - val_loss: 1.4252 - val_accuracy: 0.4426
Epoch 6/15
15/15 [==============================] - 2s 163ms/step - loss: 1.3082 - accuracy: 0.4869 - val_loss: 1.3427 - val_accuracy: 0.5246
Epoch 7/15
15/15 [==============================] - 2s 158ms/step - loss: 1.3487 - accuracy: 0.4716 - val_loss: 1.4240 - val_accuracy: 0.4426
Epoch 

In [18]:
overfit(history)

Final Training Loss: 1.1292692422866821
Final Validation Loss: 1.1111506223678589
The model is not overfitting.


In [19]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from time import time

# Define Monte Carlo Dropout CNN model with L1 regularization
def build_model():
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.001)),  # L1 regularization
        layers.Dropout(0.8),  # Regular dropout
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define Monte Carlo Dropout wrapper function
class MCDropout(tf.keras.layers.Layer):
    def __init__(self, rate):
        super(MCDropout, self).__init__()
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.keras.layers.Dropout(self.rate)(inputs, training=True) * (1 - self.rate)
        return inputs

# Instantiate the model
model = build_model()

# Wrap dropout layers with MCD behavior
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Dropout):
        layer = MCDropout(layer.rate)

# Data Augmentation (even more aggressive)
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with more aggressive data augmentation
start_time = time()
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) // 32,
                    epochs=15,  # Increase epochs for more training
                    validation_data=(x_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])  # Adjust patience

end_time = time()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')


Epoch 1/15
15/15 [==============================] - 6s 234ms/step - loss: 28.8122 - accuracy: 0.2860 - val_loss: 13.3486 - val_accuracy: 0.2295
Epoch 2/15
15/15 [==============================] - 3s 195ms/step - loss: 9.2734 - accuracy: 0.2533 - val_loss: 6.0269 - val_accuracy: 0.2459
Epoch 3/15
15/15 [==============================] - 4s 293ms/step - loss: 4.8452 - accuracy: 0.2576 - val_loss: 3.9088 - val_accuracy: 0.2295
Epoch 4/15
15/15 [==============================] - 8s 520ms/step - loss: 3.5612 - accuracy: 0.3057 - val_loss: 3.2132 - val_accuracy: 0.2951
Epoch 5/15
15/15 [==============================] - 3s 205ms/step - loss: 3.1936 - accuracy: 0.3646 - val_loss: 3.1780 - val_accuracy: 0.3770
Epoch 6/15
15/15 [==============================] - 3s 214ms/step - loss: 3.0422 - accuracy: 0.4410 - val_loss: 2.9170 - val_accuracy: 0.4426
Epoch 7/15
15/15 [==============================] - 3s 207ms/step - loss: 3.0766 - accuracy: 0.4585 - val_loss: 2.8229 - val_accuracy: 0.5082
Epoc

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from time import time

# Define Monte Carlo Dropout CNN model with L1 regularization
def build_model():
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.001)),  # L1 regularization
        layers.Dropout(0.8),  # Regular dropout
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define Monte Carlo Dropout wrapper function
class MCDropout(tf.keras.layers.Layer):
    def __init__(self, rate):
        super(MCDropout, self).__init__()
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.keras.layers.Dropout(self.rate)(inputs, training=True) * (1 - self.rate)
        return inputs

# Instantiate the model
model = build_model()

# Wrap dropout layers with MCD behavior
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Dropout):
        layer = MCDropout(layer.rate)

# Data Augmentation (even more aggressive)
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with more aggressive data augmentation
start_time = time()
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) // 32,
                    epochs=15,  # Increase epochs for more training
                    validation_data=(x_val, y_val),
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])  # Adjust patience

end_time = time()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')


Epoch 1/15
15/15 [==============================] - 34s 2s/step - loss: 30.3517 - accuracy: 0.2511 - val_loss: 13.7554 - val_accuracy: 0.2459
Epoch 2/15
15/15 [==============================] - 29s 2s/step - loss: 9.6015 - accuracy: 0.2555 - val_loss: 6.1570 - val_accuracy: 0.1639
Epoch 3/15
15/15 [==============================] - 29s 2s/step - loss: 4.8377 - accuracy: 0.2751 - val_loss: 3.8034 - val_accuracy: 0.1639
Epoch 4/15
15/15 [==============================] - 27s 2s/step - loss: 3.4784 - accuracy: 0.2969 - val_loss: 3.1265 - val_accuracy: 0.2459
Epoch 5/15
15/15 [==============================] - 27s 2s/step - loss: 3.0412 - accuracy: 0.4017 - val_loss: 2.9339 - val_accuracy: 0.3934
Epoch 6/15
15/15 [==============================] - 28s 2s/step - loss: 3.0334 - accuracy: 0.4410 - val_loss: 3.2536 - val_accuracy: 0.3607
Epoch 7/15
15/15 [==============================] - 34s 2s/step - loss: 2.9571 - accuracy: 0.4672 - val_loss: 2.7742 - val_accuracy: 0.5410
Epoch 8/15
15/15 [

In [13]:
overfit(history)

Final Training Loss: 2.4853811264038086
Final Validation Loss: 2.4593255519866943
The model is not overfitting.


**Bayes by Backpropagation**

In [14]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras import layers, losses, metrics
import numpy as np
from time import time

tfd = tfp.distributions

# Define the Bayesian Neural Network model using Bayes by Backpropagation
def build_bnn_bbb_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=input_shape),
        tfp.layers.DenseFlipout(128, activation='relu'),
        tfp.layers.DenseFlipout(64, activation='relu'),
        tfp.layers.DenseFlipout(num_classes)
    ])
    return model

# Instantiate the BNN model
input_shape = (150, 150, 3)
num_classes = 10  # Assuming 10 classes for classification
bnn_model = build_bnn_bbb_model(input_shape, num_classes)

# Define the negative log likelihood loss function for Bayesian models
neg_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

# Compile the model
bnn_model.compile(optimizer='adam', loss=neg_log_likelihood, metrics=['accuracy'])

# Train the BNN model
start_time = time()
bnn_history = bnn_model.fit(x_train, y_train,
                            batch_size=32,
                            epochs=10,
                            validation_data=(x_val, y_val))
end_time = time()

# Evaluate the BNN model on test data
test_loss, test_accuracy = bnn_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(bnn_history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(bnn_history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')


/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/layers/util.py:98: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/layers/util.py:108: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


Epoch 1/10


AttributeError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "<ipython-input-14-b7e05a9c5a40>", line 25, in None  *
        lambda x, rv_x: -rv_x.log_prob(x)

    AttributeError: 'SymbolicTensor' object has no attribute 'log_prob'


In [ ]:
overfit